In [1]:
# import relevant moduls
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read and clean files
cluster_df = pd.read_json('../../data/BTW17_Suggestions/suggestions/cluster.json')
topics_df = pd.read_json('../../data/BTW17_Twitter/lda/hashtag_topics.json')

In [3]:
cluster_df.head(3)

,x,y,z,suggestion,cluster,vector
0,-5.934507,-13.884504,-15.370424,['waz'],0,[-0.2414 0.4842 -0.1616 -0.0521 0.6159 -0.84...
1,-4.854665,-10.565660,-15.258295,"['bild', 'zeitung']",0,[-0.1158 0.48233333 -0.212 -0.3332 ...
2,-4.574384,-12.412379,-15.251037,"['interview', 'badische', 'zeitung']",0,[ 9.11500007e-02 1.71550006e-01 1.30000710e-...
3,-5.265123,-12.737933,-17.156967,['tagesanzeiger'],0,[-0.43395 0.10869999 -0.36154997 -0.757949...
4,-2.996472,-10.088203,-17.916126,['traueranzeigen'],0,[-0.9071 -0.1063 -0.4996 0.0197 -0.5739 -0.61...


In [4]:
topics_df.head(3)

,hashtag,topic_words,scores,coherence
0,dobrindt,"['dobrindt', 'dieselgate', 'fahrverbote', 'ver...","['0.082', '0.070', '0.039', '0.037', '0.035', ...",0.374704
1,ard,"['ard', 'afd', 'btw', 'zdf', 'traudichdeutschl...","['0.112', '0.101', '0.094', '0.088', '0.085', ...",0.249636
2,zeitfürmartin,"['zeitfürmartin', 'meineschule', 'gebührenfrei...","['0.171', '0.037', '0.037', '0.037', '0.025', ...",0.421646
3,ehefueralle,"['ehefueralle', 'bundestag', 'merkel', 'stimme...","['0.124', '0.017', '0.017', '0.012', '0.011', ...",0.324373
4,digitalisierung,"['digitalisierung', 'nix', 'zukunft', 'eigentl...","['0.102', '0.049', '0.030', '0.029', '0.028', ...",0.258874


In [15]:
hashtag_vectors = vectorize_hashtags(topics_df)

In [18]:
similarity_df = compare_vectors(hashtag_vectors, cluster_df)
similarity_df.head(3)

,suggestion,cluster,similarity_scores
0,[waz],0,"[0.39, 0.465, 0.393, 0.428, 0.296, 0.533, 0.29..."
1,"[bild, zeitung]",0,"[0.525, 0.605, 0.657, 0.57, 0.574, 0.516, 0.52..."
2,"[interview, badische, zeitung]",0,"[0.525, 0.511, 0.527, 0.495, 0.586, 0.509, 0.4..."


In [17]:
# save df to json
similarity_df.to_json('../../data/BTW17_Suggestions/suggestions/vector_similarity.json')